Heat Flux Calculation and first wall shaping

In [ ]:
import os
from time import time

import matplotlib.pyplot as plt

from bluemira.base.file import get_bluemira_path
from bluemira.equilibria.equilibrium import Equilibrium
from BLUEPRINT.geometry.loop import Loop
from BLUEPRINT.systems.firstwall import FirstWallDN

Loading an equilibrium file

In [ ]:
read_path = get_bluemira_path("bluemira/equilibria/test_data", subfolder="tests")
eq_name = "DN-DEMO_eqref.json"
eq_name = os.sep.join([read_path, eq_name])
eq = Equilibrium.from_eqdsk(eq_name, load_large_file=True)
x_box = [4, 15, 15, 4, 4]
z_box = [-11, -11, 11, 11, -11]
vv_box = Loop(x=x_box, z=z_box)

Calling the First Wall Class which will run
the whole first wall optimisation being everything
defined in the relevant __init__ function.
Some basic inputs need to be specified.
Particularly, if the user is dealing with a configuration
different than STEP DN, this needs to be specified.
At the moment, options alternative to STEP DN are
"DEMO_DN" and "SN".

In [ ]:
t = time()
fw = FirstWallDN(
    FirstWallDN.default_params,
    {
        "equilibrium": eq,
        "vv_inner": vv_box,
        "SN": False,
        "DEMO_like_divertor": True,
        "div_vertical_outer_target": True,
        "div_vertical_inner_target": False,
        "dx_mp": 0.001,
        "FW_optimisation": True,
    },
)

fw.build()
print(f"{time()-t:.2f} seconds")
# %%[markdown]
# The funtion "plot_hf" gives a summary plot of
# optimised wall, heat flux and flux surfaces.

In [ ]:
fig, ax = plt.subplots()
fw.plot_hf()